
#
# Agent with compositional reasoning
# LLM: OpenAI GPT4 / Mixtral (local)
# Tools: Google real-time Search, Math Calculator
# MSS 20240106
#


# Table of Contents:
## call in libs: Google Search, and OpenAI GPT4
## define tools for agent to use
## define the agent
## assign task to agent

# call in libs: Google Search, and OpenAI GPT4

In [9]:

from googleapiclient.discovery import build
from py_expression_eval import Parser
import re, time, os

# MssJupy_Agent_202401081755.ipynb : use same sysprompt with gpt4
from openai import OpenAI


print( '99' ,time.asctime( time.localtime( time.time() ) ))

99 Mon Jan  8 18:02:07 2024


In [10]:
## google api parms
with open('/home/ghtw30s/5u5uu', 'r') as file:
    gcx = file.read().rstrip()

print( '99' ,time.asctime( time.localtime( time.time() ) ))

99 Mon Jan  8 18:02:18 2024


In [12]:

with open('/home/ghtw30s/96t77i', 'r') as file:
    oaiapikey = file.read().rstrip()


print( '99' ,time.asctime( time.localtime( time.time() ) ))

99 Mon Jan  8 18:02:34 2024


In [13]:


client = OpenAI(api_key=oaiapikey)
os.environ["GOOGLE_CSE_ID"] = gcx
os.environ["GOOGLE_API_KEY"] = gkey


print( '99' ,time.asctime( time.localtime( time.time() ) ))

99 Mon Jan  8 18:02:53 2024


# force to use particular GPU

In [14]:

### force to use particular GPU

GLOBGpuToUse ="1"
# 1 being used by DemoFusion
GLOBGpuToUse ="0"



print('GLOBGpuToUse' ,GLOBGpuToUse)

import os

os.environ["CUDA_VISIBLE_DEVICES"] = GLOBGpuToUse


print(time.asctime( time.localtime( time.time() ) ))

GLOBGpuToUse 0
Mon Jan  8 18:03:58 2024


# garbage collection

In [15]:

import os, gc, re, warnings
# Garbage Collection

print(time.asctime( time.localtime( time.time() ) ))

gc.collect()


print(time.asctime( time.localtime( time.time() ) ))

Mon Jan  8 18:04:03 2024
Mon Jan  8 18:04:03 2024


# define tools for agent to use

In [21]:
#Google search engine
def search(search_term):
    search_result = ""
    service = build("customsearch", "v1", developerKey=os.environ.get("GOOGLE_API_KEY"))
    res = service.cse().list(q=search_term, cx=os.environ.get("GOOGLE_CSE_ID"), num = 10).execute()
    for result in res['items']:
        search_result = search_result + result['snippet']
    return search_result


parser = Parser()
def calculator(str):
    return parser.parse(str).evaluate({})




# MssJupy_Agent_202401061636.ipynb : replace gpt4 with local mixtral.
def calculator_rag_local(prompt):
    Calc_rag_System_prompt = """
Answer the question marked up by <Question> and </Question>.
Use the information provided in the context marked up by <Context> and </Context>.
"""
    #
    Calc_System_prompt = """
Answer the following questions and obey the following commands as best you can.

Try to estimate/calculate what the question is asking about, and respond with a number.
"""
    #    
    # 1: IR
    IRresponse = search(prompt)
    
    # 2: LLM
    user_rag_prmpt = '<Question>'+prompt +"</Question>\n\n<Context>"+ IRresponse + '</Context>'
    
    
    prompt11 = "<s>[INST]"+   Calc_rag_System_prompt +'\n\n'+ user_rag_prmpt   +"\n[/INST]\n"
    
    #
    inputs34 = tokenizer101(prompt11, return_tensors="pt")
    inputs34 = inputs34.to('cuda')
    outputs45 = model101.generate(**inputs34, max_new_tokens=9999, do_sample=False ,temperature=0.0)
    response_text = tokenizer101.decode(outputs45[0], skip_special_tokens=True)
    #
    return response_text





def calculator_rag(prompt):
    Calc_rag_System_prompt = """
Answer the question marked up by <Question> and </Question>.
Use the information provided in the context marked up by <Context> and </Context>.
"""
    #
    Calc_System_prompt = """
Answer the following questions and obey the following commands as best you can.

Try to estimate/calculate what the question is asking about, and respond with a number.
"""
    #    
    # 1: IR
    IRresponse = search(prompt)
    #print('IRresponse' , IRresponse) # debug
    
    # 2: LLM
    user_rag_prmpt = '<Question>'+prompt +"</Question>\n\n<Context>"+ IRresponse + '</Context>'
    #print('user_rag_prmpt' , user_rag_prmpt) # debug
    
    messages = [
        #{ "role": "system", "content": Calc_System_prompt },
        { "role": "system", "content": Calc_rag_System_prompt },
        { "role": "user", "content": user_rag_prmpt },
    ]
    response = client.chat.completions.create(
            model="gpt-4",
            messages=messages,
            temperature=0, # Temperature: Controls randomness, higher values increase diversity.
            max_tokens=1000,
            top_p=1, # Top-p (nucleus): The cumulative probability cutoff for token selection. Lower values mean sampling from a smaller, more top-weighted nucleus.
            # Top-k: Sample from the k most likely next tokens at each step. Lower k focuses on higher probability tokens.
        )
    response_text = response.choices[0].message.content
    
    
    #print('----------------------------------------' , time.asctime(time.localtime(time.time())) )
    ##print('* LLM response: {{{',response_text,'}}}')
    #print('* calculator_rag response: {{{',response_text,'}}}')
    
    #
    return response_text





# MssJupy_Agent_202401061636.ipynb : replace gpt4 with local mixtral.
'''
def calculator_llm(prompt):
    Calc_System_prompt = """
Answer the following questions and obey the following commands as best you can.

Try to estimate/calculate what the question is asking about, and respond with a number.
"""
    #
    messages = [
        { "role": "system", "content": Calc_System_prompt },
        { "role": "user", "content": prompt },
    ]
    response = client.chat.completions.create(
            model="gpt-4",
            messages=messages,
            temperature=0, # Temperature: Controls randomness, higher values increase diversity.
            max_tokens=1000,
            top_p=1, # Top-p (nucleus): The cumulative probability cutoff for token selection. Lower values mean sampling from a smaller, more top-weighted nucleus.
            # Top-k: Sample from the k most likely next tokens at each step. Lower k focuses on higher probability tokens.
        )
    response_text = response.choices[0].message.content
    
    print('----------------------------------------' , time.asctime(time.localtime(time.time())) )
    print('* LLM response: {{{',response_text,'}}}')
    return response_text
'''



print( '99' ,time.asctime( time.localtime( time.time() ) ))

99 Mon Jan  8 18:05:47 2024


# define LLM behavior: prompt engineering

In [22]:


# MssJupy_Agent_202401061330.ipynb : agent with compositional reasoning.

# also, it may help agent to know today's date explicitly.
    # MSS OBS 20240106: we see agent's interpretation of "the reigning men's U.S. Open champion" chnaged from from 2022 to 2023.
    # Then it might be good to explicitly declare today in the system promot!
todayis = time.asctime( time.localtime( time.time() ) )



# MssJupy_Agent_202401061636.ipynb : replace gpt4 with local mixtral.
# we removed newlines at left and rightmost of System_prompt_selfask


# MssJupy_Agent_202401081740.ipynb : edit llm system prompt: 1.break down given task, 2.self reflect (cove: chain of verification).

System_prompt_selfask = 'Today is '+todayis +""" 
Answer the following questions and obey the following commands as best you can.

You have access to the following tools:

Search: useful for when you need to answer questions about current events. You should ask targeted questions.
Calculator: Useful for when you need to answer questions that require mathematical calculations or quantitative estimations.
Response To Human: When you need to respond to the human you are talking to.

You will receive a message from the human, then you should start a loop and do one of four Options:


Option 1: Think about the subgoals for achieving the message from the human.
Sometimes human's question is a multi-hop question, and might need to be decomposed into two or more simple questions.
You then seek answer per each simpler question, and when all the simpler questions are answered then compose the answer to the initial complex multi-hop question.
The following are four examples marked up by <ComposionalReasoning> and </ComposionalReasoning> for how to decompose complex questions into sets of simpler ones:

<ComposionalReasoning>
Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali 

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball 

Question: Are both the directors of Jaws and Casino Royale from the same country? 
Are follow up questions needed here: Yes. 
Follow up: Who is the director of Jaws? 
Intermediate Answer: The director of Jaws is Steven Spielberg. 
Follow up: Where is Steven Spielberg from? 
Intermediate Answer: The United States. 
Follow up: Who is the director of Casino Royale? 
Intermediate Answer: The director of Casino Royale is Martin Campbell. 
Follow up: Where is Martin Campbell from? 
Intermediate Answer: New Zealand. 
So the final answer is: No
</ComposionalReasoning>


Option 2: You use a tool to answer the question.
For this, you should use the following format:
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: "the input to the action, to be sent to the tool"

After this, the human will respond with an observation, and you will continue.


Option 3: Reflect on how you arrived at the answer and whether the answer is correct.


Option 4: You respond to the human.
For this, you should use the following format:
Action: Response To Human
Action Input: "your response to the human, summarizing what you did and what you learned"


Begin!"""






print(System_prompt_selfask)

print( '99' ,time.asctime( time.localtime( time.time() ) ))

Today is Mon Jan  8 18:06:02 2024 
Answer the following questions and obey the following commands as best you can.

You have access to the following tools:

Search: useful for when you need to answer questions about current events. You should ask targeted questions.
Calculator: Useful for when you need to answer questions that require mathematical calculations or quantitative estimations.
Response To Human: When you need to respond to the human you are talking to.

You will receive a message from the human, then you should start a loop and do one of four Options:


Option 1: Think about the subgoals for achieving the message from the human.
Sometimes human's question is a multi-hop question, and might need to be decomposed into two or more simple questions.
You then seek answer per each simpler question, and when all the simpler questions are answered then compose the answer to the initial complex multi-hop question.
The following are four examples marked up by <ComposionalReasoning> a

# define the agent

In [24]:




# MssJupy_Agent_202401081755.ipynb : use same sysprompt with gpt4

def agent_compositional(prompt):
    #
    IterCounter = 0
    
    #
    messages = [
        { "role": "system", "content": System_prompt_selfask },
        { "role": "user", "content": prompt },
    ]
    def extract_action_and_input(text):
          action_pattern = r"Action: (.+?)\n"
        
          input_pattern = r"Action Input: \"(.+?)\""
        
          action = re.findall(action_pattern, text)
          
          action_input = re.findall(input_pattern, text, re.DOTALL)
          
          return action, action_input
    
    
    while True:
        print('\n======================================================================'  )
        print('IterCounter' ,IterCounter)
        
        print('* going to LLM...')
        response = client.chat.completions.create(
            model="gpt-4",
            messages=messages,
            temperature=0, # Temperature: Controls randomness, higher values increase diversity.
            max_tokens=1000,
            top_p=1, # Top-p (nucleus): The cumulative probability cutoff for token selection. Lower values mean sampling from a smaller, more top-weighted nucleus.
            # Top-k: Sample from the k most likely next tokens at each step. Lower k focuses on higher probability tokens.
        )
        response_text = response.choices[0].message.content
        
        print('----------------------------------------' , time.asctime(time.localtime(time.time())) )
        #
        print('{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{')
        print('* LLM response: \n' + response_text , '')
        print('}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}')
        
        
        #time.sleep(30)
        time.sleep(0)
        
        action, action_input = extract_action_and_input(response_text)
        
        # debug
        #print('DBG', 'action' ,action)
        #print('DBG', 'action_input' ,action_input)
        
        if action[-1] == "Search":
            tool = search
            
        elif action[-1] == "Calculator":
            #tool = calculator
            tool = calculator_rag
            
        elif action[-1] == "Response To Human":
            #print(f"Response: {action_input[-1]}")
            #print('----------------------------------------')
            print('========================================' , time.asctime(time.localtime(time.time())) )
            print(f"* Final: {action_input[-1]}")
            print('----------------------------------------')
            break
        
        print('----------------------------------------' , time.asctime(time.localtime(time.time())) )
        print('* TOOL chosen by the agent:' ,tool)
        observation = tool(action_input[-1])
        
        # in agent lingo, the response from a tool is called "observation"
        print('----------------------------------------', time.asctime(time.localtime(time.time())) )
        print("* Results from the tool: {{{", observation,'}}}')
        
        # mss: the llm response goes to SystemPrmopt, and the tool response goes to the UserPrompt
        messages.extend([
            { "role": "system", "content": response_text },
            { "role": "user", "content": f"Observation: {observation}" },
            ])
        
        print('----------------------------------------', time.asctime(time.localtime(time.time())) )
        # we convert to list of lists, then we flatten
        str6 = [ [key+":"+val for key, val in dct4.items()] for dct4 in messages]
        from itertools import chain
        str6f = list(chain.from_iterable(str6))
        mem44 = "\n----------------\n".join(str6f)
        print('((((((((((((((((((((((((((((((((((((((((((((((((')
        print('* memory: \n' + mem44 , '\n')
        print('))))))))))))))))))))))))))))))))))))))))))))))))')
        #
        IterCounter +=1







def agent_compositional_local(prompt, System_prompt_selfask):
    #
    IterCounter = 0
    
    prompt11 =  "<s> [INST] "+   System_prompt_selfask +'\n\n'+ prompt   +"\n[/INST]\n"
    
    
    def extract_action_and_input(text):
          action_pattern = r"Action: (.+?)\n"
        
          input_pattern = r"Action Input: (.*)"
          
          action = re.findall(action_pattern, text)
          
          action_input = re.findall(input_pattern, text)
          
          return action, action_input
    
    
    while True:
        print('\n======================================================================'  )
        print('IterCounter' ,IterCounter)
        
        print('* going to LLM...')
        
        # DEBUG
        print('prompt11=[[' , prompt11, ']]')
        
        #
        inputs34 = tokenizer101(prompt11, return_tensors="pt")
        inputs34 = inputs34.to('cuda')
        outputs45 = model101.generate(**inputs34, max_new_tokens=9999, do_sample=False ,temperature=0.0)
        response_text = tokenizer101.decode(outputs45[0], skip_special_tokens=True)
        # MSS => we HAVE TO SKIP special tokens in llm response, so that we insert <s> and </s> in correct locations in the cumulative prompt!
        
        
        print('----------------------------------------' , time.asctime(time.localtime(time.time())) )
        #
        print('{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{')
        print('* LLM response: \n' + response_text , '')
        print('}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}')
        
        
        #time.sleep(30)
        time.sleep(0)
        
        action, action_input = extract_action_and_input(response_text)
        
        if action[-1] == "Search":
            tool = search
            
        elif action[-1] == "Calculator":
            #tool = calculator
            tool = calculator_rag_local
            
            
        elif action[-1] == "Response To Human":
            #print(f"Response: {action_input[-1]}")
            #print('----------------------------------------')
            print('========================================' , time.asctime(time.localtime(time.time())) )
            print(f"* Final: {action_input[-1]}")
            print('----------------------------------------')
            break
        
        print('----------------------------------------' , time.asctime(time.localtime(time.time())) )
        print('* TOOL chosen by the agent:' ,tool)
        observation = tool(action_input[-1])
        
        # in agent lingo, the response from a tool is called "observation"
        print('----------------------------------------', time.asctime(time.localtime(time.time())) )
        print("* Results from the tool: {{{", observation,'}}}')
        
        
        
        System_prompt_selfask = response_text
        prompt = observation
        # remove any leftmost space first
        System_prompt_selfask = System_prompt_selfask.lstrip()
        prompt11 = "<s> "+   System_prompt_selfask +' </s> '       + '[INST] '+ prompt   +"\n[/INST]\n"
        
        
        
        print('----------------------------------------', time.asctime(time.localtime(time.time())) )
        #
        IterCounter +=1
        


print( '99' ,time.asctime( time.localtime( time.time() ) ))

99 Mon Jan  8 18:07:24 2024


# assign task to agent

In [31]:

user_request = "how many days till the next solar eclipse"
#import datetime




startt = time.time()
print( '11' ,time.asctime( time.localtime( time.time() ) ))

print('----------------------------------------')
print('user_request:' ,user_request)
#print('----------------------------------------')

#agent_compositional(user_request)
#agent_compositional_local(user_request, System_prompt_selfask)
agent_compositional(user_request)



#
endt = time.time()
print("FINAL: Took {} seconds to run end-to-end.".format(endt - startt ))

print( '99' ,time.asctime( time.localtime( time.time() ) ))

11 Mon Jan  8 18:13:06 2024
----------------------------------------
user_request: how many days till the next solar eclipse

IterCounter 0
* going to LLM...
---------------------------------------- Mon Jan  8 18:13:11 2024
{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{
* LLM response: 
Thought: To answer this question, I need to find out the date of the next solar eclipse and then calculate the difference between that date and today's date. 
Action: Search
Action Input: "Next solar eclipse date" 
}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}
---------------------------------------- Mon Jan  8 18:13:11 2024
* TOOL chosen by the agent: <function search at 0x7efcd462c670>
---------------------------------------- Mon Jan  8 18:13:12 2024
* Results from the tool: {{{ Otherwise all partial solar eclipses need approved filters. For approved filters, look for a certification of International Standard ISO 12312-2. The next solar ...The next annular eclipse will be October 2041. It is neve